# Imports #

In [191]:
import pandas as pd
#from openai import OpenAI
from collections.abc import MutableMapping
import json
#!pip install openai
from openai import OpenAI
import os
import openai

import pandas as pd

from PIL import Image
#!pip install opencv-python
from PIL import Image
import collections
import math

from os.path import exists




In [196]:
file_exists = exists("./DALLE_images/Aaron_Turner_13_oval_DALLE.jpg")
file_exists

True

In [198]:
file_exists = exists("./race_testing.csv")
file_exists

True

In [ ]:
# # testing if image opens properly... 
# # Imports PIL module  
# from PIL import Image 
  
# # open method used to open different extension image file 
# im = Image.open(data["filepath_og_img"][1])  
  
# # This method will show image in any image viewer  
# im.show()  

# Cleaning the CloudResearch Data 

In [120]:
#Reading response data
output_file_path = "1k_CloudResearch_study/1k_dataset_output.csv"
data = pd.read_csv(output_file_path)

#Filling additional info description box with string
data.fillna("No description provided", inplace = True)

# create the relative file_path of the image to access w/ DALLE
data["Filepath_og_img"] = data["Filename_og_img"].apply(lambda file: "1k-MTurk-Images/" + file)

In [121]:
data

,age_description,jaw_description,eyes_description,hair_description,nose_description,race_description,beard_description,gender_description,eyebrows_description,expression_description,...,Sex,Occupation Field,Relationship/Marital Status,Political Party,Gender,Country Of Residence,Household Income,Race,Employment Status,Filepath_og_img
0,40s,Rounded,Hazel,Brown short,Smaller sharp,white,None,female,Brown,smiling,...,Male,Other,Divorced,Democrat,Man,United States,"$30,000-$39,999",White,Full-time,1k-MTurk-Images/Google_1_Jane Standifer_11_ova...
1,mid 30s,"pointed, soft","small, brown, friendly","short, brown, straight","wide, small nostrils",white,"brown, scruffy, mustash",male,"brown, whispy",smiling,...,Female,Education & Training,Married,Independent,Woman,United States,"$100,000-$124,999",White,Full-time,1k-MTurk-Images/Google_1_Louis Sung_1_oval.jpg
2,90's,a round jaw,light blue eyes,short cropped hair,wide nose,black,no beard,female,thinning light brows,smiling,...,Female,Retail,In a relationship,Prefer not to say,Woman,United States,Prefer not to say,White,Full-time,1k-MTurk-Images/Google_1_Lorraine Byrd_9_oval.jpg
3,42,Roundish,Hazel,Gray,Average,white,Slight gray,male,Gray,neutral,...,Male,Other,Single,Democrat,Man,United States,"$20,000-$29,999",Black or African American,Student,1k-MTurk-Images/Adam_Koester_6_oval.jpg
4,37,Wide,Brown,Brown,Narrow,white,Shaved,male,Narrow,smiling,...,Male,Retail,Single,Something else,Man,United States,"$10,000-$19,999",White,Student,1k-MTurk-Images/Google_1_Joe Amaral_3_oval.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,mid-30s,Narrow,Dark brown,Short dark brown,Long medium size,white,None,male,Thick dark brown,smiling,...,Female,Legal,Married,Republican,Woman,United States,"$125,000-$149,999",Chinese,Unemployed,1k-MTurk-Images/Google_1_Catherine Fiske_17_ov...
996,around 40,Round,Could not tell what color but they looked dark,Brunette,Somewhat wide,hispanic,None,female,Thin and brunette,smiling,...,Female,Other,Single,Democrat,Woman,United States,"$50,000-$59,999",White,Part-time,1k-MTurk-Images/Google_1_Phillip Ledezma_17_ov...
997,mid-50s,Round,Medium brown,Short black,Large stout,black,None,male,Thin black,smiling,...,Male,Information Technology,Single,Democrat,Man,United States,"$50,000-$59,999",White,Part-time,1k-MTurk-Images/Alma_Gober_3_oval.jpg
998,late 60s,"small jawline, loose skin",piercing large blue eyes,"recieding hair line, short gray hair","bulbous snout, wide nose bridge",white,No description provided,male,thick gray,smiling,...,Male,Medicine,Single,Democrat,Man,United States,"$80,000-$89,999",Black or African American,Student,1k-MTurk-Images/Google_1_Douglas Macarthur_1_o...


In [122]:
test_data = data.iloc[1:3]

In [123]:
test_data

,age_description,jaw_description,eyes_description,hair_description,nose_description,race_description,beard_description,gender_description,eyebrows_description,expression_description,...,Sex,Occupation Field,Relationship/Marital Status,Political Party,Gender,Country Of Residence,Household Income,Race,Employment Status,Filepath_og_img
1,mid 30s,"pointed, soft","small, brown, friendly","short, brown, straight","wide, small nostrils",white,"brown, scruffy, mustash",male,"brown, whispy",smiling,...,Female,Education & Training,Married,Independent,Woman,United States,"$100,000-$124,999",White,Full-time,1k-MTurk-Images/Google_1_Louis Sung_1_oval.jpg
2,90's,a round jaw,light blue eyes,short cropped hair,wide nose,black,no beard,female,thinning light brows,smiling,...,Female,Retail,In a relationship,Prefer not to say,Woman,United States,Prefer not to say,White,Full-time,1k-MTurk-Images/Google_1_Lorraine Byrd_9_oval.jpg


# Creating DALLE Images

In [178]:
import requests
from PIL import Image
from io import BytesIO

def convert_to_jpg(url, output_path):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            image = Image.open(BytesIO(response.content))
            image.save(output_path, 'JPEG')
            print(f"Image saved as {output_path}")
            return output_path
        else:
            print("Failed to download image from URL.")
    except Exception as e:
        print(f"A error occurred: {e}")
        return "error"


In [180]:
convert_to_jpg(image_url, "testing_cat.jpg")

Image saved as testing_cat.jpg


'testing_cat.jpg'

In [126]:
# create a new folder for the target directory that contains the DALLE images if it does not exist
target_folder = "DALLE_images" 
current_directory = os.getcwd()
final_directory = os.path.join(current_directory, target_folder)
if not os.path.exists(final_directory):
   os.makedirs(final_directory)

In [129]:
import requests
from PIL import Image
from io import BytesIO

def convert_to_jpg(url, output_path):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            image = Image.open(BytesIO(response.content))
            image.save(output_path, 'JPEG')
            print(f"Image saved as {output_path}")
            return output_path
        else:
            print("Failed to download image from URL.")
    except Exception as e:
        print(f"A error occurred: {e}")
        return "error"


In [271]:
def generate_and_save_DALLE_image(row):
    prompt = ("Photo of a human face by photographer Anna-Lou Leibovitz against a white backdrop, with a camera angle of 80mm at eye level, while also containing the follwing features: " 
              +  row['gender_description'] + "; Age: " + row['age_description'] + "; Race: " 
              + row['race_description'] + "; Hair: " + row['hair_description'] + "; Eyebrows: " 
              + row['eyebrows_description'] + "; Eyes: " + row["eyes_description"] + "; Nose: " 
              + row["nose_description"] + "; Jaw: " +  row['jaw_description'] + "; Expression: " + row["expression_description"] +
    "; Beard: " + row['beard_description'] + "; Additional Info: " + row['additionalInfo_description'])

    # insert API key here
    OPENAI_API_KEY = None

    client = OpenAI(api_key = OPENAI_API_KEY)

    try: 
        # generate image for each user prompt
        response = client.images.generate(
            model="dall-e-2",
            prompt= prompt,
            size="256x256",
            quality="standard",
            n=1,
        )    
        url = response.data[0].url
    except openai.RateLimitError as error:
        # Handle the RateLimitError here
        print("Rate limit exceeded:", error)
    except: 
        print("a content error occurred")
        return("content error")

    # converting DALLE image to jpg and saving to DALLE_images folder 
    file_name = row["Filepath_og_img"]
    file_name = file_name.split("/")[1]
    new_file_name = file_name.split(".jpg")[0] + "_DALLE.jpg"
    output_path = "DALLE_images/" + new_file_name
    DALLE_file_path = convert_to_jpg(url, output_path)

    return DALLE_file_path 

In [186]:
import time

def create_DALLE_images(df):
    """
    Applies the generate_and_save_DALLE_image fx to stay w/in the openai rate limit 
    """
    # Initialize variables for rate limiting
    start_time = time.time()
    image_count = 0
    DALLE_file_paths = []
    # Iterate over each row in the dataframe
    for index, row in df.iterrows():
        DALLE_file_path = generate_and_save_DALLE_image(row)
        DALLE_file_paths.append(DALLE_file_path)
        # Apply DALL-E function to the current row
        # Replace the placeholder function call with your DALL-E function call
        # dalle_output = your_dalle_function(row['column_name'])

        # Simulate DALL-E function call with a sleep of 1 second
        time.sleep(1)

        # Update image count and check rate limit
        image_count += 1
        if image_count >3:
            # If rate limit exceeded, wait until the next minute
            elapsed_time = time.time() - start_time
            if elapsed_time < 60:
                time.sleep(60 - elapsed_time)
            # Reset image count and start time
            image_count = 0
            start_time = time.time()

        # Print progress
        print(f"Processed {index + 1} rows")
    
    # creating a new column in the df with the corresponding file paths
    df["DALLE_image_path"] = DALLE_file_paths
    print("DALLE image processing completed.")
    return df


In [149]:
test_data_df = create_DALLE_images(test_data)

a content error occurred
Processed 2 rows
Image saved as DALLE_images/Google_1_Lorraine Byrd_9_oval_DALLE.jpg
Processed 3 rows
DALLE image processing completed.


/var/folders/wz/7k6v05bx5gvffw4vkwfkmkn40000gn/T/ipykernel_66122/1370976345.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["DALLE_image_path"] = DALLE_file_paths


In [150]:
DALLE_image_df = create_DALLE_images(data)

Image saved as DALLE_images/Google_1_Jane Standifer_11_oval_DALLE.jpg
Processed 1 rows
a content error occurred
Processed 2 rows
Image saved as DALLE_images/Google_1_Lorraine Byrd_9_oval_DALLE.jpg
Processed 3 rows
Image saved as DALLE_images/Adam_Koester_6_oval_DALLE.jpg
Processed 4 rows
Image saved as DALLE_images/Google_1_Joe Amaral_3_oval_DALLE.jpg
Processed 5 rows
Image saved as DALLE_images/Google_1_Philip Patton_1_oval_DALLE.jpg
Processed 6 rows
Image saved as DALLE_images/Google_1_Stanley Kato_17_oval_DALLE.jpg
Processed 7 rows
Image saved as DALLE_images/Google_1_Teresa Birdwell_5_oval_DALLE.jpg
Processed 8 rows
Image saved as DALLE_images/Google_1_Louis Somerville_9_oval_DALLE.jpg
Processed 9 rows
Image saved as DALLE_images/Google_1_Miriam Harlow_7_oval_DALLE.jpg
Processed 10 rows
Image saved as DALLE_images/Google_1_Gerald Brotherton_5_oval_DALLE.jpg
Processed 11 rows
Image saved as DALLE_images/Google_1_Patsy Vice_5_oval_DALLE.jpg
Processed 12 rows
Image saved as DALLE_imag

UnboundLocalError: local variable 'url' referenced before assignment

In [154]:
# DALLE image generation interrupted, need to run another batch
second_batch = data[169:]

In [155]:
DALLE_second_batch = create_DALLE_images(second_batch)

a content error occurred
Processed 170 rows
Image saved as DALLE_images/Google_1_Sean Holcomb_17_oval_DALLE.jpg
Processed 171 rows
Image saved as DALLE_images/Google_1_Christopher Ruffner_1_oval_DALLE.jpg
Processed 172 rows
Image saved as DALLE_images/Google_1_Eric Mcalpine_13_oval_DALLE.jpg
Processed 173 rows
Image saved as DALLE_images/Google_1_Adam Freund_13_oval_DALLE.jpg
Processed 174 rows
Image saved as DALLE_images/Google_1_Mildred Mattern_1_oval_DALLE.jpg
Processed 175 rows
Image saved as DALLE_images/Google_1_Philip Merwin_9_oval_DALLE.jpg
Processed 176 rows
Image saved as DALLE_images/Google_1_Joseph Boston_5_oval_DALLE.jpg
Processed 177 rows
Image saved as DALLE_images/Google_1_Vincent Hamblin_3_oval_DALLE.jpg
Processed 178 rows
Image saved as DALLE_images/Google_1_Donald Normand_5_oval_DALLE.jpg
Processed 179 rows
Image saved as DALLE_images/Google_1_Darlene Perrone_11_oval_DALLE.jpg
Processed 180 rows
Image saved as DALLE_images/Google_1_Kristin Catalan_7_oval_DALLE.jpg
Pr

UnboundLocalError: local variable 'url' referenced before assignment

In [158]:
third_batch = data[188:]

In [159]:
DALLE_third_batch = create_DALLE_images(third_batch)

Image saved as DALLE_images/Google_1_Lena Clausen_12_oval_DALLE.jpg
Processed 189 rows
Image saved as DALLE_images/Google_1_Vera Becker_17_oval_DALLE.jpg
Processed 190 rows
Image saved as DALLE_images/Google_1_Douglas Bouie_11_oval_DALLE.jpg
Processed 191 rows
Image saved as DALLE_images/Google_1_Samuel Halvorson_7_oval_DALLE.jpg
Processed 192 rows
Image saved as DALLE_images/Google_1_Phillip Owensby_9_oval_DALLE.jpg
Processed 193 rows
Image saved as DALLE_images/Google_1_Adam Mckinley_7_oval_DALLE.jpg
Processed 194 rows
Image saved as DALLE_images/Google_1_Jose Mcbride_15_oval_DALLE.jpg
Processed 195 rows
Image saved as DALLE_images/Google_1_Daniel Goodman_9_oval_DALLE.jpg
Processed 196 rows
Image saved as DALLE_images/Google_1_Roger Brunelle_1_oval_DALLE.jpg
Processed 197 rows
Image saved as DALLE_images/Google_1_Jonathan Wille_10_oval_DALLE.jpg
Processed 198 rows
Image saved as DALLE_images/Angela_Bono_3_oval_DALLE.jpg
Processed 199 rows
Image saved as DALLE_images/Google_1_Flora Ja

UnboundLocalError: local variable 'url' referenced before assignment

In [185]:
fourth_batch = data[544:]
DALLE_fourth_batch = create_DALLE_images(fourth_batch)

Image saved as DALLE_images/Google_1_Douglas Cathcart_7_oval_DALLE.jpg
Processed 545 rows
Image saved as DALLE_images/Google_1_Ernest Ashburn_13_oval_DALLE.jpg
Processed 546 rows
Image saved as DALLE_images/Google_1_Belinda Dupuis_9_oval_DALLE.jpg
Processed 547 rows
Image saved as DALLE_images/Google_1_Doris Acord_7_oval_DALLE.jpg
Processed 548 rows
Image saved as DALLE_images/Google_1_Denise Eggert_1_oval_DALLE.jpg
Processed 549 rows
Image saved as DALLE_images/Google_1_Pamela Salvatore_3_oval_DALLE.jpg
Processed 550 rows
Image saved as DALLE_images/Gary_Dale_6_oval_DALLE.jpg
Processed 551 rows
Image saved as DALLE_images/Google_1_George Spurgeon_13_oval_DALLE.jpg
Processed 552 rows
Image saved as DALLE_images/Google_1_Daniel Shults_14_oval_DALLE.jpg
Processed 553 rows
Image saved as DALLE_images/Google_1_Chad Lacombe_19_oval_DALLE.jpg
Processed 554 rows
Image saved as DALLE_images/Google_1_Joann Masse_5_oval_DALLE.jpg
Processed 555 rows
Image saved as DALLE_images/Google_1_Donald Mcc

UnboundLocalError: local variable 'url' referenced before assignment

In [189]:
fourth_batch

,age_description,jaw_description,eyes_description,hair_description,nose_description,race_description,beard_description,gender_description,eyebrows_description,expression_description,...,Sex,Occupation Field,Relationship/Marital Status,Political Party,Gender,Country Of Residence,Household Income,Race,Employment Status,Filepath_og_img
544,33,Square,Large brown,long golden hair,lengthy,white,None,female,"thin, black",smiling,...,Male,Marketing and Sales,Married,Republican,Man,United States,"$70,000-$79,999",White,Full-time,1k-MTurk-Images/Google_1_Douglas Cathcart_7_ov...
545,34,double chin and rounded,Blue,Short strawberry blonde,Large and rounded,white,none,male,Patchy Blonde,smiling,...,Female,Arts,In a relationship,Democrat,Woman,United States,"$40,000-$49,999",White,Full-time,1k-MTurk-Images/Google_1_Ernest Ashburn_13_ova...
546,mid 50s,square,large brown,short brown,large and distinctive,white,None,male,thick brown,smiling,...,Male,"Science, Technology, Engineering & Mathematics",Single,Democrat,Man,United States,"$30,000-$39,999",White,Full-time,1k-MTurk-Images/Google_1_Belinda Dupuis_9_oval...
547,mid 30s,pointed jaw,"light eyes, not very big",strawberry blonde,narrow nose flared nostrils,white,none,male,sparse,neutral,...,Female,Retail,Married,Democrat,Woman,United States,"$150,0000-$174,999",White,Full-time,1k-MTurk-Images/Google_1_Doris Acord_7_oval.jpg
548,late 30's,Pointy,Deep Brown,Straight dark brown,Small and round,white,None,female,Thin round and brown,smiling,...,Female,Other,Single,Independent,Woman,United States,"$90,000-$99,999",White,Unemployed,1k-MTurk-Images/Google_1_Denise Eggert_1_oval.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,mid-30s,Narrow,Dark brown,Short dark brown,Long medium size,white,None,male,Thick dark brown,smiling,...,Female,Legal,Married,Republican,Woman,United States,"$125,000-$149,999",Chinese,Unemployed,1k-MTurk-Images/Google_1_Catherine Fiske_17_ov...
996,around 40,Round,Could not tell what color but they looked dark,Brunette,Somewhat wide,hispanic,None,female,Thin and brunette,smiling,...,Female,Other,Single,Democrat,Woman,United States,"$50,000-$59,999",White,Part-time,1k-MTurk-Images/Google_1_Phillip Ledezma_17_ov...
997,mid-50s,Round,Medium brown,Short black,Large stout,black,None,male,Thin black,smiling,...,Male,Information Technology,Single,Democrat,Man,United States,"$50,000-$59,999",White,Part-time,1k-MTurk-Images/Alma_Gober_3_oval.jpg
998,late 60s,"small jawline, loose skin",piercing large blue eyes,"recieding hair line, short gray hair","bulbous snout, wide nose bridge",white,No description provided,male,thick gray,smiling,...,Male,Medicine,Single,Democrat,Man,United States,"$80,000-$89,999",Black or African American,Student,1k-MTurk-Images/Google_1_Douglas Macarthur_1_o...


In [187]:
fifth_batch = data[584:]
DALLE_fifth_batch = create_DALLE_images(fifth_batch)

Image saved as DALLE_images/Google_1_Peter Schulman_11_oval_DALLE.jpg
Processed 585 rows
Image saved as DALLE_images/Google_1_Glenda Westbrook_5_oval_DALLE.jpg
Processed 586 rows
Image saved as DALLE_images/Google_1_Margaret Perez_15_oval_DALLE.jpg
Processed 587 rows
Image saved as DALLE_images/Google_1_Robert Koonce_17_oval_DALLE.jpg
Processed 588 rows
Image saved as DALLE_images/Google_1_Russell Kelso_11_oval_DALLE.jpg
Processed 589 rows
Image saved as DALLE_images/Google_1_Cassandra Gervais_3_oval_DALLE.jpg
Processed 590 rows
Image saved as DALLE_images/Google_1_Christopher Laskowski_7_oval_DALLE.jpg
Processed 591 rows
Image saved as DALLE_images/Aaron_Mink_9_oval_DALLE.jpg
Processed 592 rows
Image saved as DALLE_images/Google_1_Charlene Cordova_9_oval_DALLE.jpg
Processed 593 rows
Image saved as DALLE_images/Google_1_Vicky Reitz_7_oval_DALLE.jpg
Processed 594 rows
Image saved as DALLE_images/Google_1_Roger Skelton_3_oval_DALLE.jpg
Processed 595 rows
Image saved as DALLE_images/Googl

/var/folders/wz/7k6v05bx5gvffw4vkwfkmkn40000gn/T/ipykernel_66122/3872149930.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["DALLE_image_path"] = DALLE_file_paths


In [188]:
fifth_batch

,age_description,jaw_description,eyes_description,hair_description,nose_description,race_description,beard_description,gender_description,eyebrows_description,expression_description,...,Occupation Field,Relationship/Marital Status,Political Party,Gender,Country Of Residence,Household Income,Race,Employment Status,Filepath_og_img,DALLE_image_path
584,30's,"rounded jaw, bit on the bigger side","Almond shaped, brown colored, average size","brown, hard to see",A bit bigger and rounder on the nostril part,white,None,male,"Long, brown, well shaped",smiling,...,Other,Single,Democrat,Man,United States,"Less than $10,000",White,Unemployed,1k-MTurk-Images/Google_1_Peter Schulman_11_ova...,DALLE_images/Google_1_Peter Schulman_11_oval_D...
585,late 40s,warm,warm brown,dark brown with bangs,round nose,black,none,female,covered by bangs,smiling,...,Other,Single,Democrat,Woman,United States,"$10,000-$19,999",White,Part-time,1k-MTurk-Images/Google_1_Glenda Westbrook_5_ov...,DALLE_images/Google_1_Glenda Westbrook_5_oval_...
586,40's,Small,Normal green or blue,Short and blond,Small,white,None,female,Normal,smiling,...,Other,Divorced,Independent,Woman,United States,"$40,000-$49,999",White,"Not in paid work (e.g., homemaker, disabled)",1k-MTurk-Images/Google_1_Margaret Perez_15_ova...,DALLE_images/Google_1_Margaret Perez_15_oval_D...
587,early 40s,slightly pointed,regular brown,dark and short,long,white,shaved,male,bushy brown,smiling,...,Information Technology,In a relationship,Independent,Woman,United States,"Less than $10,000",Black or African American,Student,1k-MTurk-Images/Google_1_Robert Koonce_17_oval...,DALLE_images/Google_1_Robert Koonce_17_oval_DA...
588,mid to late 30s,semi-defined,brown,Short black,medium pointed nose,white,"goatee, trimmed",male,medium black,smiling,...,Arts,In a relationship,Democrat,Man,United States,"$20,000-$29,999",Filipino,Full-time,1k-MTurk-Images/Google_1_Russell Kelso_11_oval...,DALLE_images/Google_1_Russell Kelso_11_oval_DA...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,mid-30s,Narrow,Dark brown,Short dark brown,Long medium size,white,None,male,Thick dark brown,smiling,...,Legal,Married,Republican,Woman,United States,"$125,000-$149,999",Chinese,Unemployed,1k-MTurk-Images/Google_1_Catherine Fiske_17_ov...,DALLE_images/Google_1_Catherine Fiske_17_oval_...
996,around 40,Round,Could not tell what color but they looked dark,Brunette,Somewhat wide,hispanic,None,female,Thin and brunette,smiling,...,Other,Single,Democrat,Woman,United States,"$50,000-$59,999",White,Part-time,1k-MTurk-Images/Google_1_Phillip Ledezma_17_ov...,DALLE_images/Google_1_Phillip Ledezma_17_oval_...
997,mid-50s,Round,Medium brown,Short black,Large stout,black,None,male,Thin black,smiling,...,Information Technology,Single,Democrat,Man,United States,"$50,000-$59,999",White,Part-time,1k-MTurk-Images/Alma_Gober_3_oval.jpg,DALLE_images/Alma_Gober_3_oval_DALLE.jpg
998,late 60s,"small jawline, loose skin",piercing large blue eyes,"recieding hair line, short gray hair","bulbous snout, wide nose bridge",white,No description provided,male,thick gray,smiling,...,Medicine,Single,Democrat,Man,United States,"$80,000-$89,999",Black or African American,Student,1k-MTurk-Images/Google_1_Douglas Macarthur_1_o...,DALLE_images/Google_1_Douglas Macarthur_1_oval...


In [181]:
fourth_batch

,age_description,jaw_description,eyes_description,hair_description,nose_description,race_description,beard_description,gender_description,eyebrows_description,expression_description,...,Sex,Occupation Field,Relationship/Marital Status,Political Party,Gender,Country Of Residence,Household Income,Race,Employment Status,Filepath_og_img
544,33,Square,Large brown,long golden hair,lengthy,white,None,female,"thin, black",smiling,...,Male,Marketing and Sales,Married,Republican,Man,United States,"$70,000-$79,999",White,Full-time,1k-MTurk-Images/Google_1_Douglas Cathcart_7_ov...
545,34,double chin and rounded,Blue,Short strawberry blonde,Large and rounded,white,none,male,Patchy Blonde,smiling,...,Female,Arts,In a relationship,Democrat,Woman,United States,"$40,000-$49,999",White,Full-time,1k-MTurk-Images/Google_1_Ernest Ashburn_13_ova...
546,mid 50s,square,large brown,short brown,large and distinctive,white,None,male,thick brown,smiling,...,Male,"Science, Technology, Engineering & Mathematics",Single,Democrat,Man,United States,"$30,000-$39,999",White,Full-time,1k-MTurk-Images/Google_1_Belinda Dupuis_9_oval...
547,mid 30s,pointed jaw,"light eyes, not very big",strawberry blonde,narrow nose flared nostrils,white,none,male,sparse,neutral,...,Female,Retail,Married,Democrat,Woman,United States,"$150,0000-$174,999",White,Full-time,1k-MTurk-Images/Google_1_Doris Acord_7_oval.jpg
548,late 30's,Pointy,Deep Brown,Straight dark brown,Small and round,white,None,female,Thin round and brown,smiling,...,Female,Other,Single,Independent,Woman,United States,"$90,000-$99,999",White,Unemployed,1k-MTurk-Images/Google_1_Denise Eggert_1_oval.jpg
549,late 30s,square,blue,curly brunette,proportionate to face,white,None,female,light brown,smiling,...,Female,Finance,Married,Republican,Woman,United States,"$150,0000-$174,999",White,Business Owner,1k-MTurk-Images/Google_1_Pamela Salvatore_3_ov...
550,Mid 50s,"Square, rugged","Bright, blue","Wavy, short, dark brown","Angular, slightly broad",white,None,male,"Thick, dark brown",neutral,...,Female,Information Technology,In a relationship,Democrat,Woman,United States,"$80,000-$89,999",Black or African American,Student,1k-MTurk-Images/Gary_Dale_6_oval.jpg
551,Early 50,Black and white beard,Large brown wide nose,Thick black and white,Rounded high cheeks,black,Happy smile expression,male,Brown,smiling,...,Female,Information Technology,In a relationship,Democrat,Woman,United States,"$60,000-$69,999",Black or African American,Full-time,1k-MTurk-Images/Google_1_George Spurgeon_13_ov...
552,30,soft,brown,short brown,straight,white,none,male,Brown straight,smiling,...,Female,Retired,Divorced,Republican,Woman,United States,"$40,000-$49,999",White,Retired,1k-MTurk-Images/Google_1_Daniel Shults_14_oval...
553,55,square chin,"large almond shaped eyes, not protruding",mostly grey and white,"short, bulbous nose",white,No description provided,male,dark and arched,smiling,...,Female,Retired,Widowed,Republican,Woman,United States,"$20,000-$29,999",White,Student,1k-MTurk-Images/Google_1_Chad Lacombe_19_oval.jpg


In [201]:
row = data.iloc[0]

## adding column for DALLE file paths

In [214]:
def get_filepath(file_name: str):
    """
    Transforms the original filepath to the path where the DALL-E image was saved. If the file was not saved properly (due to an error), returns "error"
    Args: 
        filename: name of the original file
    Returns:
        path of the DALL-E file if found in DALLE_images directory, otherwise returns "error"
    """
    # converting DALLE image to jpg and saving to DALLE_images folder 
    print(file_name)
    file_name = file_name.split("/")[1]
    new_file_name = file_name.split(".jpg")[0] + "_DALLE.jpg"
    output_path = "DALLE_images/" + new_file_name
    if exists(output_path):
        return output_path
    else:
        return "error"

In [216]:
data["DALLE_img_filepath"] = data["Filepath_og_img"].apply(lambda filename: get_filepath(filename))

1k-MTurk-Images/Google_1_Jane Standifer_11_oval.jpg
1k-MTurk-Images/Google_1_Louis Sung_1_oval.jpg
1k-MTurk-Images/Google_1_Lorraine Byrd_9_oval.jpg
1k-MTurk-Images/Adam_Koester_6_oval.jpg
1k-MTurk-Images/Google_1_Joe Amaral_3_oval.jpg
1k-MTurk-Images/Google_1_Philip Patton_1_oval.jpg
1k-MTurk-Images/Google_1_Stanley Kato_17_oval.jpg
1k-MTurk-Images/Google_1_Teresa Birdwell_5_oval.jpg
1k-MTurk-Images/Google_1_Louis Somerville_9_oval.jpg
1k-MTurk-Images/Google_1_Miriam Harlow_7_oval.jpg
1k-MTurk-Images/Google_1_Gerald Brotherton_5_oval.jpg
1k-MTurk-Images/Google_1_Patsy Vice_5_oval.jpg
1k-MTurk-Images/Google_1_Claudia Olivarez_9_oval.jpg
1k-MTurk-Images/Google_1_Nathan Batts_15_oval.jpg
1k-MTurk-Images/Google_1_Carlos Coker_9_oval.jpg
1k-MTurk-Images/Google_1_Carl Cash_18_oval.jpg
1k-MTurk-Images/Google_1_Christina Delaney_7_oval.jpg
1k-MTurk-Images/Google_1_Latoya Hinds_9_oval.jpg
1k-MTurk-Images/Google_1_Kenneth Maggard_5_oval.jpg
1k-MTurk-Images/Google_1_Angela Chaisson_12_oval.jpg
1

# Standardizing DALLE Images

### methods to crop images and add oval to match the og image

In [261]:
from PIL import Image
import math

def crop_image(og_image_file, DALLE_image_file):
    """
    Crops the DALLE image according to the dimensions of the original image 
    Args: 
        og_image: filename of original image
        DALLE_image: filename of DALLE image

    Returns: 
        Path of new cropped DALLE image file 
    """
    

    # Open the original image
    og_image = Image.open(og_image_file)
    width_og, height_og = og_image.size

    # Open the DALLE image
    DALLE_image = Image.open(DALLE_image_file)
    width, height = DALLE_image.size

    # Calculate the dimensions for cropping
    width_diff = width - width_og
    height_diff = height - height_og
    width_diff_one_side = width_diff / 2
    height_diff_one_side = height_diff / 2

    # Calculate the cropping coordinates
    left = math.floor(width_diff_one_side)
    right = width - math.floor(width_diff_one_side)
    top = math.floor(height_diff_one_side)
    bottom = height - math.floor(height_diff_one_side)

    # Perform the cropping

    # if the original image is larger, we want to crop the original image
    if width_diff < 0 or height_diff < 0:
        cropped_img = og_image.crop((left, top, right, bottom))
        cropped_img.save(og_image_file)
        return og_image_file
    else:
        cropped_img = DALLE_image.crop((left, top, right, bottom))

    # Save the cropped image
    file_name = DALLE_image_file.split("/")[-1]  
    path = file_name.split(".")[0]
    transformed_image_path = "TRANSFORMED_DALLE_images/" + path + "_TRANSFORMED.jpg"
    cropped_img.save(transformed_image_path)
    
    return transformed_image_path


In [262]:
def get_image_pixels(image_file):
    # Open the image
    im = Image.open(image_file)

    # Load pixel data
    pix = im.load()

    # Get the width and height of the image
    width, height = im.size

    # Create a 2D array to store pixel values
    pixels_2d = [[0 for _ in range(width)] for _ in range(height)]

    # Iterate over each pixel and store its RGBA values in the 2D array
    for x in range(width):
        for y in range(height):
            pixels_2d[y][x] = pix[x, y]

    return pixels_2d


In [263]:
def create_image_from_arr(image_path, pixels_2d):
    # Assuming pixels_2d is your 2D array of pixels
    # Example pixels_2d = [[(255, 255, 255), (0, 0, 0), ...], [...], ...]

    # Assuming width and height are the dimensions of your image
    width, height = len(pixels_2d[0]), len(pixels_2d)

    # Create a new image
    new_im = Image.new("RGB", (width, height))

    # Load pixel data
    pix = new_im.load()

    # Set pixel values
    for x in range(width):
        for y in range(height):
            pix[x, y] = pixels_2d[y][x]

    # Save the image to a file
    new_im.save(image_path)
    return image_path

In [264]:
def apply_white_oval(og_image_file, AI_image_file):
    """
    Uses breadth-first search to identify the white oval frame in the og image and apply it to the AI generated image
    og_image_file, AI_image_file
    """

    # getting the 2d array of pixels from each image
    og_image_pixels = get_image_pixels(og_image_file)
    AI_image_pixels = get_image_pixels(AI_image_file)

    rows, cols = len(og_image_pixels), len(og_image_pixels[0])
    rows1, cols1 = len(AI_image_pixels), len(AI_image_pixels[1])
    
    visited = set()

    def bfs(r, c):
        q = collections.deque()
        visited.add((r, c))
        q.append((r, c))

        while q:
            row, col = q.popleft()
            directions = [[1, 0], [-1, 0], [0, 1], [0, -1]]

            for dr, dc in directions:
                r, c = row + dr, col + dc
                if (r in range(rows) 
                    and c in range(cols) 
                    and og_image_pixels[r][c] == (255, 255, 255)
                    and (r, c) not in visited):
                    q.append((r, c))
                    visited.add((r, c))
                    AI_image_pixels[r][c] = (255, 255, 255)

    # breadth first search starting from the corners of the image (to capture the oval)
    bfs(0, 0)
    bfs(rows - 1, 0)
    bfs(0, cols - 1)
    bfs(rows - 1, cols - 1)
    output_image_path = create_image_from_arr(AI_image_file, AI_image_pixels)
    return output_image_path

In [265]:
def transform_DALLE_image(og_image_file, DALLE_image_file):
    """
    Standardizes the DALLE image by cropping to fit dimensions of original image and applying the white oval
    """
    cropped_image = crop_image(og_image_file, DALLE_image_file)
    transformed_image_path = apply_white_oval(og_image_file, cropped_image)
    return transformed_image_path

### implementing methods

In [268]:
# create a new folder for the target directory that contains the transformed images if it does not exist
target_folder = "TRANSFORMED_DALLE_images" 
current_directory = os.getcwd()
final_directory = os.path.join(current_directory, target_folder)
if not os.path.exists(final_directory):
   os.makedirs(final_directory)

In [269]:
data["DALLE_images_transformed"] = data.apply(lambda row: transform_DALLE_image(row["Filepath_og_img"], row["DALLE_img_filepath"]) 
                                                        if row["DALLE_img_filepath"] != "error" 
                                                        else row["DALLE_img_filepath"], axis = 1)

### writing final transformed dataframe to csv

In [270]:
data.to_csv("final_output.csv", mode="w", index=False)